In [53]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

# Load Models

In [54]:
facenet = cv2.dnn.readNet('models/deploy.prototxt', 'models/res10_300x300_ssd_iter_140000.caffemodel')
# Face Detection Model Load
model = load_model('models/mask_detector.model')
# Mask Detection Model Load

# Load Video

In [55]:
cap = cv2.VideoCapture('imgs/04.mp4') # read Video VideoCapture(0) -> notebook Webcam Open
ret, img = cap.read() # 재생되는 비디오의 한 프레임씩 읽는다. -이곳에서 초기화
# 프레임을 제대로 읽었다면 ret 값이 True가 됨
# 읽은 프레임은 img
# cap이 제대로 초기화 되지 않았을 경우 에러코드 리턴, cap.isOpened() 함수를 이용해 cap이 초기화가 제대로 되었는지 확인 가능
if cap.isOpened():
    print("cap이 초기화 되었습니다.")

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
#fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
out = cv2.VideoWriter('output.mjpg', fourcc, cap.get(cv2.CAP_PROP_FPS), (img.shape[1], img.shape[0]))


cap이 초기화 되었습니다.


# Detect Mask

In [56]:
while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    h, w = img.shape[:2] # image shape 높이 , 너비 값 지정

    blob = cv2.dnn.blobFromImage(img, scalefactor=1., size=(300, 300), mean=(104., 177., 123.))
    # blobFromImage를 통해서 dnn 모듈이 사용하는 형태로 이미지를 변형한다. axis순서만 바뀜
    facenet.setInput(blob) # facenet의 input으로 변경
    dets = facenet.forward() # 결과를 추론 -> Detection 결과가 dets에 저장
    result_img = img.copy()
    #result_img = cv2.resize(img, (500, 500), fx=0.3, fy=0.7, interpolation=cv2.INTER_AREA) 
    for i in range(dets.shape[2]):
        # dets에 저장된 객체를 for문을 돌며 체크(여러개의 얼굴이 detection 될 수 있다)
        confidence = dets[0, 0, i, 2]
        if confidence < 0.5: # 50%미만이면 continue
            continue

        x1 = int(dets[0, 0, i, 3] * w)
        y1 = int(dets[0, 0, i, 4] * h)
        x2 = int(dets[0, 0, i, 5] * w)
        y2 = int(dets[0, 0, i, 6] * h)
        # x와 y의 바운딩 박스 설정
        
        face = img[y1:y2, x1:x2] # 얼굴 부분만 잘라내기

        face_input = cv2.resize(face, dsize=(224, 224)) #resize 224,224
        face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB) # cvtColor -> BGR값을RGB로 변환
        face_input = preprocess_input(face_input) # 모바일넷v2와 같이 preprocessing
        # 현재 상태 (224, 224, 3)
        # input 값에서는 (1, 224, 224, 3)의 형태로 넣어주여야 하므로 np.expand_dims()로 자원을 추가
        face_input = np.expand_dims(face_input, axis=0)
        
        # mask 쓴 확률 안쓴 확률 저장
        mask, nomask = model.predict(face_input).squeeze()

        if mask > nomask:
            color = (0, 255, 0) # BRG (cv2에선 BRG순으로 입력값을 받음)
            label = 'Mask %d%%' % (mask * 100)
        else:
            color = (0, 0, 255)
            label = 'No Mask %d%%' % (nomask * 100)
    
        # 사각형 그리기
        # cv2.rectangle(img , start, end, color, thickness)
        # img - 그림을 그릴 이미지 , start -> 시작 좌표, end -> 종료 좌표, color -> BRG값, thickness-> 선의 두께 -1이면 안쪽을 채움 
        cv2.rectangle(result_img, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=color, lineType=cv2.LINE_AA)
        # cv2.putText(img, text, org, font, fontSacle, color)
        # img - image , text -> 표시할 문자열, org - 문자열이 표시될 위치
        cv2.putText(result_img, text=label, org=(x1, y1 - 10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=color, thickness=2, lineType=cv2.LINE_AA)

    out.write(result_img)
    winname = 'test'
    cv2.namedWindow(winname)
    cv2.moveWindow(winname, 0, 0)
    cv2.imshow(winname, result_img)
    if cv2.waitKey(1) == ord('q'):
        cv2.destroyAllWindows()
        break
out.release()
cap.release()